In [2]:
import pandas as pd
import numpy as np
import re
import itertools

# What do I do?

I import the data, I group it by District and Verkiezingsdatum (so as to have unique groups), and then I calculate positions. 

Then I calculate the winning and losing candidates by taking "if <position> > <Aantal zetels>" then Loser, otherwise winner. 
    
Then, I calculate the margins for the losers to the lowest winner. 
    
Finally, I make a new dataset containing only the closely elected politicians and the names of the winners of the district.

In [3]:
data = pd.read_csv("../Data/elections/election_results_details.csv", encoding='latin-1')
data = data.drop('Unnamed: 0', axis=1)

## First, make a counter

In [4]:
def counter(group):
    temp = pd.Series(range(0,len(group)), dtype='int')
    temp = temp + 1
    return temp

In [5]:
data["counts"] = data.groupby(['District', 'Verkiezingdatum'], sort = False)['Naam'].transform(counter)

## Then, make win indicators

In [6]:
data['Aantal zetels'] = pd.to_numeric(data['Aantal zetels'], errors='coerce')

data["won"] = np.where(data["counts"] <= data['Aantal zetels'], 1,0)

## Now, calculate the margin to the lowest winner

In [7]:
winners = data.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False).apply(lambda x: x[x['won'] == 1])
winners['Aantal stemmen'] = pd.to_numeric(winners['Aantal stemmen'])

In [8]:
minwinners = winners.loc[winners.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False)['Aantal stemmen'].idxmin()]
minwinners = minwinners.reset_index().iloc[:,2:]

# Now

Make a per unique election min_procentueel and min_aantal_stemmen

Then merge them with the large dataframe with all candidates

And calculate the margn

If negative, then they lost. We then proceed to check a couple of things

In [9]:
#Make a per unique election min_procentueel and min_aantal_stemmen
minwinners = minwinners.iloc[:,[4,5,2,3]]

minwinners = minwinners.rename({'Aantal stemmen':'min_stem_win', 
                  'Procentueel':'min_pct_win'}, axis=1)

In [10]:
#Then merge them with the large dataframe with all candidates
allmgn = data.merge(minwinners, how='left', left_on=['District','Verkiezingdatum'], right_on=['District','Verkiezingdatum'])

In [11]:
#Calculate the margin
allmgn['Aantal stemmen'] = pd.to_numeric(allmgn['Aantal stemmen'], errors='coerce')

def p2f(x):
    var = []
    for i in x:
        var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
    return var

allmgn['Procentueel'] = p2f(allmgn['Procentueel'])

In [12]:
def p2f_nan(x):
    var = []
    for i in x:
        if type(i) == str:
            var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
        else:
            var.append(i)
    return var

allmgn['min_pct_win'] = p2f_nan(allmgn['min_pct_win'])

In [13]:
allmgn['margin'] = allmgn['Procentueel'] - allmgn['min_pct_win']

In [14]:
# If negative, they lost:

def close_cand(x, pct):
    var = []
    for i in x:
        var.append(pd.to_numeric(np.where(i < 0 and i > -pct, 1, 0)))
    return pd.Series(var)
        

allmgn['close_cand_5pct'] = close_cand(allmgn['margin'], 0.05)
allmgn['close_cand_10pct'] = close_cand(allmgn['margin'], 0.10)
allmgn['close_cand_20pct'] = close_cand(allmgn['margin'], 0.20)

In [15]:
allmgn['Kiesdrempel'] = pd.to_numeric(allmgn['Kiesdrempel'], errors='coerce')
allmgn['Aantal stemmen geldig'] = pd.to_numeric(allmgn['Aantal stemmen geldig'], errors='coerce')

#allmgn[allmgn['Aantal stemmen geldig'] > allmgn['Kiesdrempel']]
#Everything is larger than kiesdrempel

## Finally..

Take the data frame, take only the close losers

- For each close loser, look up the (just) winner in that election (only 1 person)
- Match each loser to that winner (multiple matches per district possible if e.g. 2 close losing candidates)
    - Merge strategy: take all the eligible just losers
    - Merge them to a filtered dataset finished with the just winners on the basis of district, date (left join with left = just losers)



In [16]:
# All just Losers
justlosers = allmgn[allmgn['close_cand_20pct'] == 1]


# Filtered dataset with just winners
justwinners = allmgn[allmgn['counts'] == allmgn['Aantal zetels']]

# Filtered dataset with all winners
allwinners = allmgn[allmgn['counts'] <= allmgn['Aantal zetels']]

In [17]:
#change the varnames for the justwinners to avoid duplicate issues
justwinners = justwinners.reset_index().iloc[:,1:7].rename({
    'Naam':'Naam_win',
    'Aanbevolen door':'Aanbevolen door_win',
    'Aantal stemmen':'Aantal stemmen_win',
    'Procentueel':'Procentueel_win'}, axis=1)

In [18]:
# Merge justwinners with just losers
closepairs = justlosers.merge(justwinners, how='left', on=['District','Verkiezingdatum'])

# Retrieve some additional information

- Are they on parlement.com?
- Sterfplaats
- Sterfteleeftijd (geboortejaar)
- Distance to Den Haag via Google Maps API

In [19]:
#J'ai utilise ce fichier auparavant pour générer data_entry.csv sur Google Drive
#Maintenant j'en fais un fichier plus précis, quitte à omettre quelques matches potentiels 

#closepairs.to_csv("../Administration/close_pairs.csv")

politici = allwinners['Naam'].drop_duplicates()
#politici = politici.drop_duplicates()

closepairs['alrdy_in_pols'] = np.where(closepairs['Naam'].isin(politici),'ja','nee')

closepairs2 = closepairs[closepairs['alrdy_in_pols'] == 'nee']

In [190]:
closepairs2.to_csv("../Administration/close_pairs2.csv")

## Make a new sheet with non-close pairs to enlarge the dataset


(24-03-2021): The dataset of close elections is almost complete. Hence, we enlarge the scope of the dataset to more (non-close or less close) elections. This is implemented in the following code. We then write a cv file to the Administration folder, nonclose_pairs, and use another .py script to make that into an easily-accessible data-entry file. 

In [44]:
allmgn['close_cand_30pct'] = close_cand(allmgn['margin'], 0.30)

possiblenewones = allmgn[(allmgn['close_cand_20pct'] == 0) & (allmgn['close_cand_30pct'] == 1)]

#From this set, make sure only to have the losers, not the politicians
#Also, make sure to have no overlap with the entry_file dataset in names (import it here, and filter on that basis)

In [45]:
# import the "first" dataset and extract the names
algehad = pd.read_csv("../Administration/new_data_entry_file.csv")['Naam'].unique()

# find out which names are also in possiblenewones, and filter the possiblenewones dataset on that basis
possiblenewones['algehad'] = np.where(possiblenewones['Naam'].isin(algehad), 'ja', 'nee')
possiblenewones = possiblenewones[possiblenewones['algehad'] == 'nee']
# also filter the possiblenewones data on the condition that there be no politicians from the file
possiblenewones['isapolitician'] = np.where(possiblenewones['Naam'].isin(politici), 'ja', 'nee')

lesscloserpairs = possiblenewones[possiblenewones['isapolitician'] == 'nee']

/home/bas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [46]:
lesscloserpairs.to_csv("../Administration/less_close_pairs.csv")

In [ ]:
# Now, continue in make_data_entry_file_lessclose.ipynb